In [1]:
# librairie globales
import io
from io import StringIO
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pandas

# pour convert_pdf_to_txt
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

# pour prepare_sentence
import nltk, os, subprocess, code, glob, re, traceback, sys, inspect
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize


[nltk_data] Downloading package wordnet to /home/thomas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/thomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/thomas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/thomas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [87]:
class extract_pdf:
    
    def __init__(self, path, langage="french"):
        self.path = path
        self.allfiles = [f for f in listdir(self.path) if isfile(join(self.path, f))]
        print(self.allfiles)
        self.langage = langage
        
        
    def convert_pdf_to_txt(self):

        pdf_folder = Path(self.path)
        # print(self.allfiles)
        list_txt = list()
        for files in self.allfiles:
            rsrcmgr = PDFResourceManager()
            retstr = StringIO()
            codec = 'utf-8'
            laparams = LAParams()
            device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
            print(files)
            fp = open(pdf_folder / files, 'rb')
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            password = ""
            maxpages = 0
            caching = True
            pagenos = set()
            #text = files
            for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching,
                                          check_extractable=True):
                interpreter.process_page(page)
            
            text = retstr.getvalue()
            print("Longueur du doc : ")
            print(len(text))
            print("Début du doc : ")
            print(text[1:100])
            list_txt.append(text)
            print(len(list_txt))
            fp.close()
            device.close()
            retstr.close()

        return list_txt
      
        
    
    def prepare_sentence(self, list_txt):
        
        stop_words = set(stopwords.words(self.langage))
        clean_text_list = list()
        #Retrait de la ponctuation
        #text = re.sub('[^a-zA-Z]', ' ', sentence)
        for sentence in list_txt:
            print("Longueur du doc de base : ")
            print(len(sentence))
            print("Début du doc de base : ")
            print(sentence[1:100])
            #Conversion en minuscule
            text = sentence.lower()

            #Retrait des tags
            text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

            #Retrait des numéros et caractères spéciaux
            text=re.sub("(\\d|\\W)+"," ",text)

            #Conversion en liste
            text = text.split()


            text = [word for word in text if not word in  
                    stop_words] 
            text = " ".join(text)
            print("Longueur du doc : ")
            print(len(text))
            print("Début du doc : ")
            print(text[1:100])
            clean_text_list.append(text)
        return clean_text_list
    
    def export_text(self, list_txt):
        
        for sentence in list_txt:
            text_file = open("./sample.txt", "w")
            n = text_file.write(sentence)
            text_file.close()
        
        

In [22]:
toto = extract_pdf('/home/thomas/Documents/mediaData/echantillons_pdf/')

['20190927_PAR.pdf', '20190930_PAR.pdf']


In [23]:
toto2 = toto.convert_pdf_to_txt()

20190927_PAR.pdf
Longueur du doc : 
1345542
Début du doc : 
html><head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
</head><body>
<span
1
20190930_PAR.pdf
Longueur du doc : 
862894
Début du doc : 
html><head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
</head><body>
<span
2


In [24]:
type(toto2[0])
toto2[0]


'<html><head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n</head><body>\n<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:652px; height:864px;"></span>\n<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>\n<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:586px; top:80px; width:32px; height:8px;"><span style="font-family: b\'BYUTLS+TodaySHOP-Medium\'; font-size:8px">PUBLICITÉ\n<br></span></div><span style="position:absolute; border: black 1px solid; left:586px; top:83px; width:40px; height:9px;"></span>\n<span style="position:absolute; border: black 1px solid; left:33px; top:89px; width:585px; height:782px;"></span>\n<span style="position:absolute; border: black 1px solid; left:33px; top:89px; width:585px; height:782px;"></span>\n<span style="position:absolute; border: black 1px solid; left:33px; top:89px; width:585px; height:782px;"></span>\n<div style="position:absolute; bor

In [14]:
toto3 = toto.prepare_sentence(toto2)

Longueur du doc de base : 
184182
Début du doc de base : 
b'html><head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n</head><body>\n<span'


TypeError: cannot use a string pattern on a bytes-like object

In [16]:
toto4 = toto.export_text(toto2)

TypeError: write() argument must be str, not bytes

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, XMLConverter, HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO
import re

In [21]:
class extract_pdf:
    
    def __init__(self, path, langage="french"):
        self.path = path
        self.allfiles = [f for f in listdir(self.path) if isfile(join(self.path, f))]
        print(self.allfiles)
        self.langage = langage
        
        
    def convert_pdf_to_txt(self):

        pdf_folder = Path(self.path)
        # print(self.allfiles)
        list_txt = list()
        for files in self.allfiles:
            rsrcmgr = PDFResourceManager()
            retstr = BytesIO()
            codec = 'utf-8'
            laparams = LAParams()
            device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
            print(files)
            fp = open(pdf_folder / files, 'rb')
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            password = ""
            maxpages = 0
            caching = True
            pagenos = set()
            #text = files
            for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching,
                                          check_extractable=True):
                interpreter.process_page(page)
            
            text = retstr.getvalue()
            text =text.decode('utf-8')
            print("Longueur du doc : ")
            print(len(text))
            print("Début du doc : ")
            print(text[1:100])
            list_txt.append(text)
            print(len(list_txt))
            fp.close()
            device.close()
            retstr.close()

        return list_txt
      
        
    
    def prepare_sentence(self, list_txt):
        
        stop_words = set(stopwords.words(self.langage))
        clean_text_list = list()
        #Retrait de la ponctuation
        #text = re.sub('[^a-zA-Z]', ' ', sentence)
        for sentence in list_txt:
            print("Longueur du doc de base : ")
            print(len(sentence))
            print("Début du doc de base : ")
            print(sentence[1:100])
            #Conversion en minuscule
            text = sentence.lower()

            #Retrait des tags
            text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

            #Retrait des numéros et caractères spéciaux
            text=re.sub("(\\d|\\W)+"," ",text)

            #Conversion en liste
            text = text.split()


            text = [word for word in text if not word in  
                    stop_words] 
            text = " ".join(text)
            print("Longueur du doc : ")
            print(len(text))
            print("Début du doc : ")
            print(text[1:100])
            clean_text_list.append(text)
        return clean_text_list
    
    def export_text(self, list_txt):
        
        for sentence in list_txt:
            text_file = open("./sample.txt", "w")
            n = text_file.write(sentence)
            text_file.close()
        
        

In [11]:
u = toto2[0].decode('latin1')


In [12]:
type(u)

str

In [3]:
u[0:1000]

NameError: name 'u' is not defined

In [125]:
re.findall( r'HelveticaNeueLTStd-Md\'; font-size:41px">(.*?)\n<br></span></div>', u,re.DOTALL)


['La capitale Â« en deuil Â»',
 'Un Paris plus  \n<br>ou moins rÃ©ussi ',
 'Une icÃ´ne du cool Ã\xa0 la franÃ§aise',
 'Lâ\x80\x99hÃ©ritage du grand Jacques ',
 'Â« Câ\x80\x99Ã©tait un pÃ¨re de famille pour nous Â»',
 'JournÃ©e noire Ã\xa0 Rouen',
 'Sont-ils prÃªts Ã\xa0 en faire plus ?',
 'A lâ\x80\x99Ã©cole de \n<br>Â« The Politician Â»',
 'Le monsieur te demandaitâ\x80¦',
 'Boyer, une graine de star',
 'Â« Il nâ\x80\x99arrivait pas  \n<br>Ã\xa0 tenir la coupe Â»']

In [127]:
re.findall( r'DINOT-Regular\'; font-size:11px">(.*?)</span><span style="font-family: b\'BYVFCF+', u,re.DOTALL)

['La nouvelle a cueilli les Ã©lus, jeudi \n<br></span><span style="font-family: b\'BYVGQJ+DINOT-Regular\'; font-size:11px">matin, en pleine confÃ©rence de presse \n<br>en amont du Conseil de Paris. Jacques \n<br>Chirac, acteur omniprÃ©sent de quatre \n<br>dÃ©cennies de la vie politique franÃ§aise \n<br>et internationale, est mort Ã\xa0 son domi-\n<br>cile parisien Ã\xa0 lâ\x80\x99Ã¢ge de 86 ans. Lâ\x80\x99HÃ´tel \n<br>de Ville, oÃ¹ le premier maire de la ca-\n<br>pitale a pris ses quartiers en 1977, se \n<br>met soudain en branle. Entre larmes, \n<br>souvenirs et moments de battement. \n<br>Les Ã©lus de la droite parisienne se suc-\n<br>cÃ¨dent au micro : en tÃªte, Marie-Claire \n<br>CarrÃ¨re-GÃ©e, fraÃ®chement Ã©lue prÃ©si-\n<br>dente du groupe LRI (Les RÃ©publicains \n<br>et IndÃ©pendants). Cette ancienne se-\n<br>crÃ©taire gÃ©nÃ©rale adjointe de lâ\x80\x99ElysÃ©e \n<br>sous Chirac et conseillÃ¨re quand il Ã©tait \n<br>maire rend spontanÃ©ment hommage \n<br>Ã\xa0 cette Â« ï¬\x81gure 

In [17]:
test = b'Jacques Chirac avait quitt\xc3\xa9'

In [20]:
toto2[0].decode('utf-8')

'<html><head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n</head><body>\n<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:652px; height:864px;"></span>\n<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>\n<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:586px; top:80px; width:32px; height:8px;"><span style="font-family: b\'BYUTLS+TodaySHOP-Medium\'; font-size:8px">PUBLICITÉ\n<br></span></div><span style="position:absolute; border: black 1px solid; left:586px; top:83px; width:40px; height:9px;"></span>\n<span style="position:absolute; border: black 1px solid; left:33px; top:89px; width:585px; height:782px;"></span>\n<span style="position:absolute; border: black 1px solid; left:33px; top:89px; width:585px; height:782px;"></span>\n<span style="position:absolute; border: black 1px solid; left:33px; top:89px; width:585px; height:782px;"></span>\n<div style="position:absolute; bor